# ................................................به نام حضرت دوست  .................................................... 

در این گزارش که همراه کد هست قرار است به وسیله مدل عصبی 

fully connected

داده های ژنوم را دسته‌بندی کنیم  و به ۶ کلاس مرتبط داده شده نسبت دهیم



In [1]:
import keras
from keras.models import Sequential
from keras.layers import LSTM ,Bidirectional ,Dense,Activation ,Input
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import tensorflow as tf
from typing import List
from typing import List
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from pathlib import Path

In [2]:
! pip install gensim

در سلول زیر تابعی نوشته شده است که از یک استرینگ زیرجمله های 

k-mer

را برمیگرداند که در واقع از این ها به عنوان کلمات در این دنباله هایی که داریم میتوانیم در نظر بگیریم
طول این کلمات را در ادامه ۱۰ داده ایم در واقع داریم کلمات ۱۰ کارکتری را از دنباله استرینگی استخراج میکنیم

In [3]:
def build_kmers(sequence:str, k:int)->List[str]:
    """
        inputs:
                sequence: a sting that represent a DNA
                k: parameter to produce k-mer list
        return: list of kmers
    """
    kmers = []
    n_kmers = len(sequence) - k + 1

    for i in range(n_kmers):
        kmer = sequence[i:i + k]
        kmers.append(kmer)

    return kmers

##  خواندن داده آموزش
به وسیله پکیج پانداز از فایل اکسل داده ها را میخوانیم سپس یک ستون به این داده ها که نماینده طول هر توالی هست را در میسازیم

In [4]:
df = pd.read_csv("./data/training_set.csv")
df["len"] =df["Sequence"].apply(lambda x:len(x))
df

,Type,Sequence,len
0,Class5,TACCACCTACGCTGACAATGGATGTTATTGTACCCGATTCGAATTA...,10735
1,Class5,TGTTTATGTATCCGCCCATAAATAGCTTAGCGTTGACTACCGCTTG...,10737
2,Class3,TCAATATAGGCCACGGGCAACCCGTTGATATTTTGTGTGACGGTTG...,29409
3,Class6,TAGTTTTAGTAACGTTCCTCTCCGTTCTCAATTTATTCTTTTATTG...,2341
4,Class2,TCGCGTGCCGGGGCGACCGCGCCAGTATTGCCCATCCCCTTGATTG...,30096
...,...,...,...
1315,Class5,TACCACCTACGCTGACAATGGATGTTATTGTACCCGTTTCGAATTA...,10735
1316,Class6,TAGTTTTAGTAAGTTTGGTCCCATTCAATCACGTTCGGATGCGCTG...,2341
1317,Class1,GCTGGGTAATTTTAGGTTGGTTGGCGATCGCGCCACTATCGCACCG...,29350
1318,Class5,TACCACCTACGCTGACAATGGATGTTATTGTACCCGATTCGAATTA...,10735


در ادامه میبینیم که به طور میانگین طول توالی های ما ۲۰هزار است  

In [5]:
df["len"].mean()

20146.206818181818

در این قسمت از تابع بالا استفاده میکنیم و به ازای هر توالی لیست 

k-mers

آن را اضافه میکنیم
تا همانطور که در ادامه توضیح داده خواهد شد از آن اسنفاده کنیم


In [6]:
df["k-mers"] = df["Sequence"].apply(lambda x :build_kmers(x,5))
df

,Type,Sequence,len,k-mers
0,Class5,TACCACCTACGCTGACAATGGATGTTATTGTACCCGATTCGAATTA...,10735,"[TACCA, ACCAC, CCACC, CACCT, ACCTA, CCTAC, CTA..."
1,Class5,TGTTTATGTATCCGCCCATAAATAGCTTAGCGTTGACTACCGCTTG...,10737,"[TGTTT, GTTTA, TTTAT, TTATG, TATGT, ATGTA, TGT..."
2,Class3,TCAATATAGGCCACGGGCAACCCGTTGATATTTTGTGTGACGGTTG...,29409,"[TCAAT, CAATA, AATAT, ATATA, TATAG, ATAGG, TAG..."
3,Class6,TAGTTTTAGTAACGTTCCTCTCCGTTCTCAATTTATTCTTTTATTG...,2341,"[TAGTT, AGTTT, GTTTT, TTTTA, TTTAG, TTAGT, TAG..."
4,Class2,TCGCGTGCCGGGGCGACCGCGCCAGTATTGCCCATCCCCTTGATTG...,30096,"[TCGCG, CGCGT, GCGTG, CGTGC, GTGCC, TGCCG, GCC..."
...,...,...,...,...
1315,Class5,TACCACCTACGCTGACAATGGATGTTATTGTACCCGTTTCGAATTA...,10735,"[TACCA, ACCAC, CCACC, CACCT, ACCTA, CCTAC, CTA..."
1316,Class6,TAGTTTTAGTAAGTTTGGTCCCATTCAATCACGTTCGGATGCGCTG...,2341,"[TAGTT, AGTTT, GTTTT, TTTTA, TTTAG, TTAGT, TAG..."
1317,Class1,GCTGGGTAATTTTAGGTTGGTTGGCGATCGCGCCACTATCGCACCG...,29350,"[GCTGG, CTGGG, TGGGT, GGGTA, GGTAA, GTAAT, TAA..."
1318,Class5,TACCACCTACGCTGACAATGGATGTTATTGTACCCGATTCGAATTA...,10735,"[TACCA, ACCAC, CCACC, CACCT, ACCTA, CCTAC, CTA..."


# wordEmbedding
دلیل استفاده ما از از این روش این است که ما به ازای هر کلمه یک بردار نماینده داشته باشیم  میتوانیم از این روش استفاده کنیم این روش با در نظر گرفتن پنجره هایی توالی کلمه ها را در نظر میگیرد و مفهوم این کلمات را در کل  میسنجد و بدین شکل  به ازای هر کلمه یک بردار در نظر میگیرد 
پس ما برای ساخت بردار نماینده هر یک از این کلمات نیاز داریم کل کلمات هر توالی را به عنوان  ورودی به کلاس زیر میدهیم 
سایز بردار را ۲۰۰ در نظر گرفته ایم
این کلاس زیر مدل را میسازد و کش میکند که اگر دوباره ابجکتی از این کلاس ساخته شد از مدلی که قبلا ساخته شده استفاده کند
اما متد زیر چه کاری انجام میدهد؟؟

def embed()...

در این متد ما لیستی از کلمات را میدهیم سپس این متد بردار هر یک را حساب کرده و به وسیله  روش

TfIdf

میانگین وزن دار میگیرد و یک بردار به عنوان بردار خروجی باز میگرداند
## چرا از

## TfIdf

## ?استفاده میکنیم

زیرا اگر میانگین معمولی از وکتور ها بگیریم به عنوان بردار نماینده توالی به خوبی تاثیر هر کلمه را در نظر نگرفته ایم
برای مثال ما کلماتی که در همه توالی ها هستند و زیاد تکرار شده اند را به یک اندازه با بقیه میانگین گرفته ایم درصورتی که این توالی ها تاثیرر کمتری دارند زیرا همه جا هستند و در تصمیمگیری نهایی نقش کمتری دارند
ولی در عوض وزن بیشتری به کلمات خاصتر و کم تکرارتر میدهیم

## خروجی چیست؟
پس با استفاده از دو روش بالا به  ازای هر کلمه برداری در نظر گرفتیم و  با گرفتن میانگین وزندار از این بردارها به ازای هر توالی داده شده ما یک بردار داریم که برای دسته بندی میتوانیم از آن استفاده کنیم
حال در ادامه با استفاده از یک شبکه عصبی فولی کانکتد عملیات دسته بندی را انجام میدهیم


In [7]:
class DocToVec:
    
    def __init__(self, dataset:List[List] , vec_size = 200 , model_path = 'word2vec.model'):
        """ Here we train our model and calculate tf-idf weights.

        Args:
            dataset (List[List]): Each item of this list is a list of tokens obtained from a document.
            vec_size (int, optional): The size of the vector of each document. Defaults to 200.
            model_path (str, optional): The path of pre-trained model. If there is no file in the specified 
            path, It trains a model with the dataset and saves the trained model. Defaults to 'word2vec.model'.
        """

        self.vec_size = vec_size
        
        
        
        if not Path(model_path).is_file():
            print('There is no pre-trained model. Going to train a model ...')
            self.wordToVecModel = Word2Vec(
                window = 10,
                min_count=2,
                workers=4,
                vector_size = self.vec_size
            )
        
            self.wordToVecModel.build_vocab(dataset)
            
            self.wordToVecModel.train(
                dataset,
                total_examples = self.wordToVecModel.corpus_count,
                epochs = 20,
            )
            
            self.wordToVecModel.save(model_path)
            
        else:
            
            print('Loading the model ...')
            self.wordToVecModel = Word2Vec.load(model_path)
        
        con_train_data = [" ".join(a) for a in dataset]
        
        self.tfIdfVectorizer=TfidfVectorizer(use_idf=True,
                                        dtype = np.float64,
                                        lowercase = False,
                                        vocabulary = self.wordToVecModel.wv.index_to_key)
        
        self.tfIdfVectorizer.fit(con_train_data)
        
        dictionary = self.tfIdfVectorizer.get_feature_names()
        self.dictToNum = {d:i for i,d in enumerate(dictionary)}
        
        
    def embed(self, tokens:List):
        """Maps the input to a vector

        Args:
            tokens (List): List of tokens (don't forget to do preprocessing before 
            extracting tokens)

        Returns:
            Numpy Array: The vector of document
        """
        
        ti = self.tfIdfVectorizer.transform([" ".join(tokens)])
        
        weights = np.squeeze(np.asarray(ti[0].T.todense()))
        
        vec = np.zeros((self.vec_size,))
        sum_weights = 0
        for k in set(tokens):
            try:
                word_vec = self.wordToVecModel.wv[k]
                weight = weights[self.dictToNum[k]]
                vec += word_vec * weight
                sum_weights += weight
            except KeyError:
                pass
        vec /= sum_weights
        return vec
        
        
        
        

حال در اینجا با استفاده از کلاس بالا به مدل امبدینگ خود را آموزش میدهیم و به یک ستون بردار به ازای هر توالی به دیتافریم خود اضافه میکنیم

In [8]:
word_to_vec_model = DocToVec(dataset=df["k-mers"].to_list())
df["vec"]= df["k-mers"].apply(lambda x:word_to_vec_model.embed(x))
df

Loading the model ...


/Users/hoseinlook/Projects/uni/GLOBAL_UNI_VENV/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Type,Sequence,len,k-mers,vec
0,Class5,TACCACCTACGCTGACAATGGATGTTATTGTACCCGATTCGAATTA...,10735,"[TACCA, ACCAC, CCACC, CACCT, ACCTA, CCTAC, CTA...","[0.14347037438654303, -0.09826718607041093, -0..."
1,Class5,TGTTTATGTATCCGCCCATAAATAGCTTAGCGTTGACTACCGCTTG...,10737,"[TGTTT, GTTTA, TTTAT, TTATG, TATGT, ATGTA, TGT...","[0.15608204759534353, -0.10743600239410199, -0..."
2,Class3,TCAATATAGGCCACGGGCAACCCGTTGATATTTTGTGTGACGGTTG...,29409,"[TCAAT, CAATA, AATAT, ATATA, TATAG, ATAGG, TAG...","[0.22348955288315472, -0.045752539952623295, 0..."
3,Class6,TAGTTTTAGTAACGTTCCTCTCCGTTCTCAATTTATTCTTTTATTG...,2341,"[TAGTT, AGTTT, GTTTT, TTTTA, TTTAG, TTAGT, TAG...","[0.12861910830795006, -0.06960499866601161, -0..."
4,Class2,TCGCGTGCCGGGGCGACCGCGCCAGTATTGCCCATCCCCTTGATTG...,30096,"[TCGCG, CGCGT, GCGTG, CGTGC, GTGCC, TGCCG, GCC...","[0.2680618561736469, -0.021119162201327892, 0...."
...,...,...,...,...,...
1315,Class5,TACCACCTACGCTGACAATGGATGTTATTGTACCCGTTTCGAATTA...,10735,"[TACCA, ACCAC, CCACC, CACCT, ACCTA, CCTAC, CTA...","[0.09989383920623382, -0.08351448598232104, -0..."
1316,Class6,TAGTTTTAGTAAGTTTGGTCCCATTCAATCACGTTCGGATGCGCTG...,2341,"[TAGTT, AGTTT, GTTTT, TTTTA, TTTAG, TTAGT, TAG...","[0.1354878669723666, -0.08694451862251948, -0...."
1317,Class1,GCTGGGTAATTTTAGGTTGGTTGGCGATCGCGCCACTATCGCACCG...,29350,"[GCTGG, CTGGG, TGGGT, GGGTA, GGTAA, GTAAT, TAA...","[0.16401321047208844, -0.08347763841540236, 0...."
1318,Class5,TACCACCTACGCTGACAATGGATGTTATTGTACCCGATTCGAATTA...,10735,"[TACCA, ACCAC, CCACC, CACCT, ACCTA, CCTAC, CTA...","[0.12067560817271122, -0.07888817984440853, -0..."


در سلول زیر داده‌های ورودی که برای آموزش مورد نیاز هست  با لیبلشان از دیتافریم بیرون میکشیم و  به آرایه های نامپای تبدیل میکنیم 

In [9]:
Y= df["Type"].to_list()
X = np.array(df["vec"].to_list())

# change to np
le = preprocessing.LabelEncoder()
Y =le.fit_transform(Y)

Y.shape

(1320,)

داده های خود را به دو داده آموزش و ارزیابی تقسیم بندی میکنیم تا مدل بدست امده را ارزیابی کنیم

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [11]:
X_train.shape

(884, 200)

حال داده های ما برای مدل اماده شده اند شبکه عصبی ساده ای در نظر میگیریم  چون ورودی های وکتور های ۲۰۰ تایی هستند پس  

input -> 200

و دو لایه مخفی در نظر میگیریم و از تابع فعال سازی 

relu

و لایه اخر را نیز   به طول ۶ در نظر میگیرییم زیرا ۶ کلاس بیشتر نداریم
و از آپتیمایز آدام استفاده کرده ایم زیرا بج سایز را کم در نظر گرفته ایم ۳۲

In [12]:
model =Sequential()
model.add(Input(shape=(200,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(6))


Metal device set to: Apple M1 Pro


2022-07-04 03:16:46.112613: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-04 03:16:46.112725: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
model.fit(X_train,Y_train, batch_size=32 , epochs=10)

Epoch 1/10


2022-07-04 03:16:46.239489: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-04 03:16:46.391839: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


28/28 [==============================] - 0s 6ms/step - loss: 1.3806 - accuracy: 0.6324
Epoch 2/10
28/28 [==============================] - 0s 5ms/step - loss: 0.6896 - accuracy: 0.8575
Epoch 3/10
28/28 [==============================] - 0s 5ms/step - loss: 0.3032 - accuracy: 0.9977
Epoch 4/10
28/28 [==============================] - 0s 6ms/step - loss: 0.1414 - accuracy: 0.9977
Epoch 5/10
28/28 [==============================] - 0s 6ms/step - loss: 0.0755 - accuracy: 0.9977
Epoch 6/10
28/28 [==============================] - 0s 5ms/step - loss: 0.0385 - accuracy: 0.9989
Epoch 7/10
28/28 [==============================] - 0s 5ms/step - loss: 0.0234 - accuracy: 1.0000
Epoch 8/10
28/28 [==============================] - 0s 5ms/step - loss: 0.0154 - accuracy: 1.0000
Epoch 9/10
28/28 [==============================] - 0s 5ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 10/10
28/28 [==============================] - 0s 5ms/step - loss: 0.0083 - accuracy: 1.0000


In [15]:
test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=2)
print('Test accuracy:', test_acc)

14/14 - 0s - loss: 0.0070 - accuracy: 1.0000 - 135ms/epoch - 10ms/step
Test accuracy: 1.0


2022-07-04 03:16:48.088333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


همانطور که دیدیم مدل ما روی داده های تستی که از تقسیم بندی به دست آمده بود به خوبی عمل کرده است
حال از داده های دولوپ استفاده میکنیم تا مدل را ارزیابی کنیم


In [16]:
dev_df = pd.read_csv("./data/development_set.csv")
dev_df["k-mers"] = dev_df["Sequence"].apply(lambda x :build_kmers(x,5))
dev_df['vec']=dev_df['k-mers'].apply(lambda x:word_to_vec_model.embed(x))
dev__vecs = np.array(dev_df["vec"].to_list())
output= model.predict(dev__vecs)
results = list(le.inverse_transform(np.argmax(output,axis=1)))
dev_df["predicted"]=pd.Series(results)

2022-07-04 03:16:50.142286: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [22]:
accuracy = sum(dev_df["Type"]==dev_df["predicted"])/len(dev_df)
print(F"accuracy_on_development_dataset= {accuracy*100}%")

accuracy_on_development_dataset= 100.0%


باز هم مدل روی این داده ها خوب عمل کرده و دقت ۱۰۰ درصد را دارد
حال به سراغ داده های تست میرویم تا آنهارا پیشبینی کنیم

In [18]:
test_df = pd.read_csv("./data/test_set.csv")
test_df

,Sequence
0,TACTACCTACGCTGACAATGGATGTTATTGTACCCGATTCGAATTA...
1,TACTACCTACGCTGACAATGGATGTTATTGTACCCGATTCGAATTA...
2,TCCTTTAACCCTCTGGCCGGGTAACTTGTTTGGTTGGTTGCCCGAT...
3,TCTCCTAACCCCCTGGCTGGGTAATTTTAGGTTGGTTGGCGATCGC...
4,GTTTGGTCCCATTCAATCACGTTCGGATGCGCTGCACCGCCTTTTA...
...,...
395,GTAATTTTAGGTTGGTTGGCGATCGCGCCACTATCGCACCGCGCTT...
396,GTTTGGTTGGTTGCCCGATCGCGCCACTATCGCACCGCGCTTTGAT...
397,ATCCCTTACATTCTAGCCAAGCTCGCGTGCCCGGGCGACCGCGCCA...
398,TACTACCTACGCTGACAATGGATGTTATTGTACCCGATTCGAATTA...


داده های تست همانند داده های آموزش میخوانیم و همان عملیاتی را که روی داده های آمورش انجام دادیم را روی آن انجام میدهیم و سپس به مدل میدهیم و نتیجه را برمیگردانیم

In [19]:
test_df["k-mers"] = test_df["Sequence"].apply(lambda x :build_kmers(x,5))
test_df['vec']=test_df['k-mers'].apply(lambda x:word_to_vec_model.embed(x))
test_vecs = np.array(test_df["vec"].to_list())
output= model.predict(test_vecs)
results = list(le.inverse_transform(np.argmax(output,axis=1)))

In [20]:
print(results)

['Class5', 'Class5', 'Class3', 'Class1', 'Class6', 'Class1', 'Class6', 'Class3', 'Class4', 'Class1', 'Class4', 'Class3', 'Class2', 'Class4', 'Class4', 'Class3', 'Class6', 'Class3', 'Class3', 'Class3', 'Class4', 'Class1', 'Class5', 'Class6', 'Class1', 'Class5', 'Class3', 'Class3', 'Class3', 'Class3', 'Class1', 'Class3', 'Class6', 'Class1', 'Class6', 'Class1', 'Class3', 'Class5', 'Class5', 'Class6', 'Class5', 'Class4', 'Class5', 'Class5', 'Class3', 'Class6', 'Class4', 'Class5', 'Class1', 'Class2', 'Class4', 'Class3', 'Class4', 'Class3', 'Class3', 'Class6', 'Class1', 'Class6', 'Class5', 'Class3', 'Class1', 'Class3', 'Class3', 'Class3', 'Class3', 'Class6', 'Class4', 'Class5', 'Class6', 'Class6', 'Class6', 'Class3', 'Class2', 'Class6', 'Class1', 'Class1', 'Class4', 'Class6', 'Class6', 'Class2', 'Class5', 'Class4', 'Class1', 'Class1', 'Class3', 'Class5', 'Class6', 'Class4', 'Class3', 'Class5', 'Class3', 'Class1', 'Class1', 'Class6', 'Class5', 'Class3', 'Class5', 'Class1', 'Class4', 'Class1',

In [21]:
len(results)

400